#### 정상작동 확인
- 사용데이터 : 선관위 API 리턴값 유권자 데이터
- 기능 : 선관위 데이터와 주민등록인구통계를 합쳐 총선 속성데이터(인구, 선거인수) 출력

In [5]:
from selenium import webdriver 
import time
import pandas as pd
import urllib.request
import requests
from bs4 import BeautifulSoup
import csv
import os
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}
import numpy as np
import xmltodict
import json
pd.set_option('display.max_rows', 10000)

In [6]:
url = 'http://apis.data.go.kr/9760000/ElcntInfoInqireService/getElpcElcntInfoInqire'
servicekey = 'F6CN7k8VOt7oHN2xhXc8tEXhBJbH7lR1qMpFxCEfWNAA3P34sS/WiDexjmujb7s6lWnuAnq0UEktsxTGsi2FIw=='

li = []
#page number: 1~6
for i in range(1,7):
    params ={'serviceKey' : servicekey, 'pageNo' : f'{i}', 'numOfRows' : '100', 'sgId' : '20200415', 'sgTypecode' : '2', 'sggName' : '', 'sdName' : '' }
    response = requests.get(url, params=params, headers=headers)
    #응답값 xml 형태로 만들기
    dict = xmltodict.parse(response.content)
    jsonstring = json.dumps(dict['response']['body'],ensure_ascii=False)
    jsonobj = json.loads(jsonstring)
    num = int(jsonobj['items']['item'][-1]['num'])
    # num = 100
    # print(jsonobj['items']['item'][-1]['sggName'])
    #후보자 정보를 담을 df_선거구 만들기
    coldict = {'sdName':'시도','sggName':'선거구',
               'wiwName':'시군구',
                'ppltCnt':'인구수', 'cfmtnElcnt':'확정선거인수'}
    df_선거구 = pd.DataFrame(columns=coldict.keys(), index = list(range(num)))
    #반복문을 이용하여 데이터프레임으로 만들기
    for col in df_선거구.columns:
        try:
            for j in range(num):
                df_선거구.loc[j, col]=jsonobj['items']['item'][j][col]
        except:
            pass
    df_선거구.rename(coldict, inplace = True, axis = 1)
    li.append(df_선거구)
df_선거구 = pd.concat(li).dropna()
df_선거구 = df_선거구.loc[df_선거구['시군구'] != '합계'].reset_index(drop = True)

# 인구수 = api 
df_선거구['시도_선거구'] = df_선거구['시도'] + '_' +df_선거구['선거구']
df_선거구['인구수'],df_선거구['확정선거인수']  = df_선거구['인구수'].astype(int),df_선거구['확정선거인수'].astype(int)
df_sgg = df_선거구.groupby(['시도_선거구'])[['인구수','확정선거인수']].sum().reset_index()
df_sgg['시도'] = df_sgg['시도_선거구'].str.split('_').str[0]
df_sgg['선거구'] = df_sgg['시도_선거구'].str.split('_').str[1]
df_sgg = df_sgg[['시도','선거구','인구수','확정선거인수','시도_선거구']]

In [7]:
df_sgg = df_sgg.sort_values('인구수', ascending= False)

In [8]:
df_code = pd.read_excel('./adm_code.xls')
#세종시 없애고 행정구 빼버리기
df_code.columns = ['SIDO_CD','SIDO_NM','SIGUNGU_CD','SIGUNGU_NM','ADM_CD','AMD_NM']
df_code = df_code.iloc[1:,:]
df_code['SIDO_CD'] = df_code['SIDO_CD'].astype(str)
df_code.head()
df_code['SIGUNGU_NM'] = df_code['SIGUNGU_NM'].apply(lambda x: '세종특별자치시' if x == '세종시' else x) # 세종시 없애기
#행정구빼버리기
#df_code_1 = df_code 2,3,4 concat

#df_code_4의 경우 행정구역을 split 한 값 중 2번째 값을 사용
#level2,3,4으로부터 분할
df_code['행정구역_level'] = df_code['SIGUNGU_NM'].apply(lambda x: len(x.split()))
df_code_1 = df_code.loc[df_code['행정구역_level'] == 1]
df_code_2 = df_code.loc[df_code['행정구역_level'] == 2]

df_code_1['SIGUNGU_NM'] = df_code_1['SIGUNGU_NM'].str.split().str[0]
df_code_2['SIGUNGU_NM'] = df_code_2['SIGUNGU_NM'].str.split().str[0]
df_code = pd.concat([df_code_1,df_code_2], axis = 0, ignore_index=True)

#행정구빼버리기
#df_code_1 = df_code 2,3,4 concat
#df_code_4의 경우 행정구역을 split 한 값 중 2번째 값을 사용
#level2,3,4으로부터 분할
df_code['행정구역_level'] = df_code['SIGUNGU_NM'].apply(lambda x: len(x.split()))
df_code_1 = df_code.loc[df_code['행정구역_level'] == 1]
df_code_2 = df_code.loc[df_code['행정구역_level'] == 2]

df_code_1['SIGUNGU_NM'] = df_code_1['SIGUNGU_NM'].str.split().str[0]
df_code_2['SIGUNGU_NM'] = df_code_2['SIGUNGU_NM'].str.split().str[0]
df_code = pd.concat([df_code_1,df_code_2], axis = 0, ignore_index=True)

df_code.loc[df_code['행정구역_level'] == 2]
df_code['SIDO_SIGUNGU_NM'] = df_code['SIDO_NM'] + '_' + df_code['SIGUNGU_NM']
df_code['SIDO_SIGUNGU_CD'] = df_code['SIDO_CD'] + df_code['SIGUNGU_CD']
#시도, 시군구 코드 관계 시리즈 만들기
li_sido = df_code[['SIDO_CD','SIDO_NM']].drop_duplicates().set_index('SIDO_CD')
li_sidosigungu = df_code[['SIDO_SIGUNGU_CD','SIDO_SIGUNGU_NM']].drop_duplicates().set_index('SIDO_SIGUNGU_CD')
# li_sido = df_code[['SIDO_CD','SIDO_NM']].unique().set_index('SIDO_CD', axis = 1)

/var/folders/_p/b25fmdzs0gs_7l54_j6744sm0000gn/T/ipykernel_15249/3641767897.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_code_1['SIGUNGU_NM'] = df_code_1['SIGUNGU_NM'].str.split().str[0]
/var/folders/_p/b25fmdzs0gs_7l54_j6744sm0000gn/T/ipykernel_15249/3641767897.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_code_2['SIGUNGU_NM'] = df_code_2['SIGUNGU_NM'].str.split().str[0]


In [9]:
dic_sd = {'경기도':'31','전라북도':'35','경상남도':'38','충청남도':'34','인천광역시':'23',
          '경상북도':'37','부산광역시':'21','서울특별시':'11','충청북도':'33','전라남도':'36',
          '대전광역시':'25','대구광역시':'22','광주광역시':'24','제주특별자치도':'39','강원도':'32',
          '울산광역시':'26','세종특별자치시':'29'}
df_sgg['SIDO_CD'] = df_sgg['시도'].map(dic_sd)

In [11]:
regdate = input('인구기준연월을 입력하세요 ex)2024년01월:')
sgg_nm = input('선거이름을 입력하세요 ex)21대총선:')
df_pop = pd.read_csv(f'../general_election/유권자/{sgg_nm}/{regdate}_연령별인구현황_1세.csv', encoding = 'cp949', thousands=',')
#df_pop:->시도, 시군구 통계 제외, 18세 미만을 거르기 위한 1세단위 데이터

#행정동 단위로 처리: 시도, 시군구 단위의 통계 삭제
#df_pop_1: 20대 이상 인구수를 10세 단위로 사용하기 위한 데이터
pop = df_pop.loc[~df_pop['행정구역'].str.contains('00000')]
df_pop_1 = pd.read_csv(f'../general_election/유권자/{sgg_nm}/{regdate}_연령별인구현황_10세.csv', encoding='cp949', thousands=',')
df_pop_1 = df_pop_1.loc[~df_pop_1['행정구역'].str.contains('00000')].iloc[:,5:14]
# 70세 이상 인구수는 하나의 필드로 합치기
df_pop_1[f'{regdate}_계_70세이상'] = df_pop_1[[f'{regdate}_계_70~79세', f'{regdate}_계_80~89세', 
                        f'{regdate}_계_90~99세',f'{regdate}_계_100세 이상']].sum(axis=1)
# 합친 필드값 드롭
df_pop_1 = df_pop_1.drop([f'{regdate}_계_70~79세',f'{regdate}_계_80~89세', 
                          f'{regdate}_계_90~99세', f'{regdate}_계_100세 이상'], axis = 1)
# pop에서는 행정구역(코드포함) 및 18, 19세 데이터 가져오기
pop = pop[['행정구역',f'{regdate}_계_18세',f'{regdate}_계_19세']]

pop = pd.concat([pop, df_pop_1], axis = 1)
li_col = [f'{regdate}_계_18세', f'{regdate}_계_19세', f'{regdate}_계_20~29세',
       f'{regdate}_계_30~39세', f'{regdate}_계_40~49세', f'{regdate}_계_50~59세',
       f'{regdate}_계_60~69세', f'{regdate}_계_70세이상']

#출장소를 포함하는 행의 인덱스를 이용, 이전행(관할 읍면동)의 나이대별 인구수와 합치기
li_출장 = pop.loc[pop['행정구역'].str.contains('출장소')].index
#출장소의 인구수는 해당 출장소의 윗행에 추가되어 있음
for i in li_출장:
    for c in li_col:
        pop.loc[i-1,c] = pop.loc[i-1,c] + pop.loc[i,c] 

pop = pop.loc[~pop['행정구역'].str.contains('출장소')] #출장소 60개소 제거

#행정구역코드 별도 컬럼으로 구성 for 행정구역 레벨을 표시하기 위함
pop['행정구역코드'] = pop['행정구역'].str.split('(').str[1].str.strip(')')
pop['행정구역'] = pop['행정구역'].str.split('(').str[0]
#행정구역 레벨을 별도 컬럼으로 구성
pop['행정구역_level'] = pop['행정구역'].apply(lambda x: len(x.split()))
#행정구역 3단계로 맞춰주기 1) level이 2인 개체를 3으로 조정
#->전부 세종특별 자치시에 해당
#시도-시군구-읍면동 분할 시, 시군구 컬럼을 별도 컬럼으로 생성필요
pop.loc[pop['행정구역_level']==2]['행정구역']
#행정구역 3단계로 맞춰주기 2) level이 4인 개체를 3으로 조정
#->수원,성남, 안양, 안산, 고양, 용인, 청주, 천안, 전주, 포항, 창원
#시도-시군구-읍면동 분할 시, 구 정보를 별도 컬럼으로 구성할 필요 있음(나머디 시군구는 null처리)
# pop.loc[pop['행정구역_level']==4]['행정구역']
# level 2,3,4 모두 시도: 0, 시군구: 1 자리값을 가짐
# 시도, 시군구 컬럼생성
#행정구컬럼추가, level_2, 3 == 0, level_4 == 추구 별도 추가
pop['시도'] = pop['행정구역'].str.split().str[0]
pop['시군구'] = pop['행정구역'].str.split().str[1]
pop['행정구'] = 0
#level2,3,4으로부터 분할
pop_2 = pop.loc[pop['행정구역_level'] == 2]
pop_3 = pop.loc[pop['행정구역_level'] == 3]
pop_4 = pop.loc[pop['행정구역_level'] == 4]
#level2 = 세종
#시군구 컬럼 추가 시 세종특별자치시(현재 시도 값)라고 표기
pop_2['시군구'] = pop_2['시도']
#pop_3의 경우 행정구역을 split 한 값 중 2번째 값을 사용
pop_3['시군구'] = pop_3['행정구역'].str.split().str[1]
# pop_3[['시도','시군구']]
#pop_4의 경우 행정구역을 split 한 값 중 2번째 값을 사용
pop_4['시군구'] = pop_4['행정구역'].str.split().str[1]
pop_4['행정구'] = pop_4['행정구역'].str.split().str[2]
# pop_4[['시도','시군구','행정구']].head()
#pop_1 = pop 2,3,4 concat
pop_1 = pd.concat([pop_2,pop_3,pop_4], axis = 0, ignore_index=True)
pop_1['읍면동'] = pop_1['행정구역'].str.split().str[-1]
# pop_1['읍면동'].head(20)
#행정동의'제'에 대한 처리를 해야함
#모든 '제'가 아닌 다음글자로 숫자가 오는 '제'만 제거해야함
#읍면동명 중 '.' 은 선거구의 표기와 맞추기 위해 '・'로 수정
pop_1['분동여부']=pop_1.loc[pop_1['읍면동'].str.contains('제')]['읍면동'].str.split('제').str[1].str[0]
pop_1['읍면동'] = pop_1['읍면동'].apply(lambda x: x.replace('.','·'))
print(len(pop_1))
pop_2 = pop_1.loc[pop_1['분동여부'].isin(['1','2','3','4','5','6','7','8','9'])]
pop_3 = pop_1.loc[~pop_1['분동여부'].isin(['1','2','3','4','5','6','7','8','9'])]

pop_2['읍면동'] = pop_2['읍면동'].str.replace('제','')
pop_1 = pd.concat([pop_2,pop_3], axis = 0).drop('분동여부', axis = 1)
# pop_1['읍면동'] = pop_1['읍면동'].str.replace(aa,'·')

# pop['cor'] = pop['구시군명'] + pop['읍면동']
# pop_1.head()
#나이에 대한 처리: 0세 ~ 17세: 컬럼위치로 처리
# 인구 마지막 컬럼은 70대 이상으로 합치고
# 18, 19세 유권자는 20대에 합치기
# -> pop_1을 나누어 컬럼명 처리하고 다시 붙이기
pop_행정 = pop_1[['행정구역','시도','시군구','행정구','읍면동','행정구역코드']]
pop_인구 = pop_1[pop_1.columns.difference(list(pop_행정.columns))].drop(['행정구역_level'],axis = 1)
pop_인구['20'] = pop_인구[[f'{regdate}_계_18세', f'{regdate}_계_19세', f'{regdate}_계_20~29세']].sum(axis = 1)
pop_인구 = pop_인구.drop([f'{regdate}_계_18세', f'{regdate}_계_19세', f'{regdate}_계_20~29세'], axis = 1)
pop_인구.columns = ['age_30', 'age_40', 'age_50',
       'age_60', 'age_70', 'age_20']

pop_2 = pd.concat([pop_행정, pop_인구], axis = 1)
pop_4 = pop_2.copy()
pop_4['읍면동'] = pop_4['읍면동'].apply(lambda x: x.replace('제제','제') if '제제' in x else x)
pop_4.columns = ['행정구역', '시도', '시군구', '행정구', '읍면동', '행정구역코드', 'age_20', 'age_30', 'age_40',
       'age_50', 'age_60', 'age_70']
pop_4.head(10)

3553


/var/folders/_p/b25fmdzs0gs_7l54_j6744sm0000gn/T/ipykernel_15249/2810925173.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_2['시군구'] = pop_2['시도']
/var/folders/_p/b25fmdzs0gs_7l54_j6744sm0000gn/T/ipykernel_15249/2810925173.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_3['시군구'] = pop_3['행정구역'].str.split().str[1]
/var/folders/_p/b25fmdzs0gs_7l54_j6744sm0000gn/T/ipykernel_15249/2810925173.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,행정구역,시도,시군구,행정구,읍면동,행정구역코드,age_20,age_30,age_40,age_50,age_60,age_70
36,서울특별시 종로구 창신제1동,서울특별시,종로구,0,창신1동,1111067000,600,496,834,993,776,575
37,서울특별시 종로구 창신제2동,서울특별시,종로구,0,창신2동,1111068000,886,793,1654,1495,1225,1017
38,서울특별시 종로구 창신제3동,서울특별시,종로구,0,창신3동,1111069000,912,802,1168,1092,847,896
39,서울특별시 종로구 숭인제1동,서울특별시,종로구,0,숭인1동,1111070000,817,811,1087,995,861,793
40,서울특별시 종로구 숭인제2동,서울특별시,종로구,0,숭인2동,1111071000,1910,1030,1534,1404,1127,2594
52,서울특별시 중구 신당제5동,서울특별시,중구,0,신당5동,1114065000,1816,1410,1743,1481,1294,1573
60,서울특별시 용산구 원효로제1동,서울특별시,용산구,0,원효로1동,1117056000,4179,2655,2185,1805,1293,3057
61,서울특별시 용산구 원효로제2동,서울특별시,용산구,0,원효로2동,1117057000,2340,1934,2238,1860,1653,2101
65,서울특별시 용산구 이촌제1동,서울특별시,용산구,0,이촌1동,1117063000,2905,4181,4371,3115,3052,3031
66,서울특별시 용산구 이촌제2동,서울특별시,용산구,0,이촌2동,1117064000,1283,1117,1305,1280,1208,936


In [12]:
len(pop_4)

3553

In [14]:
pop_4.to_csv(f'../general_election/유권자/{sgg_nm}/{regdate}_총선_인구.csv', index = False, encoding = 'utf-8')

In [198]:
# df_sgg[['SIDO_CD','SIDO_NM']].to_csv('../유권자/21대총선_시도코드_시도.csv', index = False, encoding = 'cp949')
# df_sgg.drop('pop_sum', axis = 1).to_csv('../유권자/21대총선_api_연령별_인구_선거구.csv', index = False, encoding = 'cp949')
# df_선거구.to_excel('../유권자/21대총선_api_시군구_선거구매칭.xlsx', index = False)